In [ ]:
import dt_apriltags as apriltag
import cv2
import annotation_utils as utils

import numpy as np
from scipy.spatial.transform import Rotation as R

import pyrealsense2 as rs
import urx
np.set_printoptions(suppress=True)

In [ ]:
dist = np.array([[-0.04797802,  0.04744357,  0.00017416,  0.00067967, -0.00408397]])
detector = apriltag.Detector(families="tagStandard52h13")
mtx = np.array([[633.09029639, 0., 629.06462963], [0., 638.7544391, 362.74013262], [0., 0., 1.]])

camera_params = [635.0, 635.0, 629.0646296262861, 362.7401326185789]

def solve_pnp(obj_points, imagePoints,mtx, dist):
    success, rvec, tvec,inliers  = cv2.solvePnPRansac(obj_points, np.array([imagePoints]), mtx, dist,reprojectionError = 0.5)
        
    if not success:
        print('not success in PnP')
        return
    return tvec, rvec,inliers


In [53]:
img_with_marker_name = 'marker.jpg'

img = cv2.imread(img_with_marker_name)

res = utils.detect_apriltag(img, camera_params)

corners_2d = res[0].corners
center_2d = res[0].center

marker_pts_2d = np.vstack((center_2d, corners_2d))
corners_3D = np.array(
[(0, 0,0),
 (24, 24,0),
 (24, -24,0),
 (-24, -24,0),
 (-24,24, 0)], dtype=float)
# print(marker_pts_2d)
# print(corners_3D)
tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,marker_pts_2d,mtx, dist)
print("inliers_m\n",inliers_m.T)

rot_error = R.from_rotvec(rvec_gt.flatten()).as_matrix() @ R.from_rotvec(rvec_m.flatten()).as_matrix().T

tf_m_in_cam = np.eye(4)
tf_m_in_cam[:3,:3] = R.from_rotvec(rvec_m.flatten()).as_matrix()
tf_m_in_cam[:3,3] = tvec_m.flatten()
print(tf_m_in_cam)

inliers_m
 [[0 1 2 3 4]]
[[ -0.99988473   0.01309331  -0.00768735 -14.69300341]
 [ -0.01307967  -0.9999128   -0.00182186  -4.2235037 ]
 [ -0.00771053  -0.0017211    0.99996879 287.77684282]
 [  0.           0.           0.           1.        ]]


In [51]:
anchor_gt_2D = np.array(
[(597, 349),
 (579, 325),
 (614, 324),
 (562, 350),
 (633, 349),
 (581, 380),
 (615, 379)], dtype=float)

img_with_marker_name = 'marker.jpg'
img_without_marker_name = 'no_marker.jpg'

keypoints, anchors = utils.get_points_from_CAD()
# anchors[:,2] = 2.25
tvec_gt,rvec_gt,inliers_gt = solve_pnp(anchors,anchor_gt_2D,mtx, dist)
print("inliers_gt\n",inliers_gt.T)

tf_s_in_c = np.eye(4)
tf_s_in_c[:3,:3] = R.from_rotvec(rvec_gt.flatten()).as_matrix()
tf_s_in_c[:3,3] = tvec_gt.flatten()
print(tf_s_in_c)

inliers_gt
 [[0 1 2 3 6]]
[[ -0.99906004  -0.02333392   0.03653156 -14.7314117 ]
 [  0.02743588  -0.99285334   0.11614435  -6.48518631]
 [  0.03356038   0.11703746   0.99256029 286.14096793]
 [  0.           0.           0.           1.        ]]


In [55]:
# доворот
z_from = tf_s_in_c[:3,2]
z_to = tf_m_in_cam[:3,2]
angle = np.arccos(z_from@z_to)
rot_ax = np.cross(z_from,z_to)

# print(angle,rot_ax)
rot_vec = rot_ax /np.linalg.norm(rot_ax) * angle
rot_mtx = R.from_rotvec(rot_vec).as_matrix()
# print(rot_mtx @ z_from -z_to )

tf_s_in_c[:3,:3]  = rot_mtx @ tf_s_in_c[:3,:3]

# корректировка по OZ
tf_s_in_c[2,3] = tf_m_in_cam[2,3]
print(tf_s_in_c)


[[ -0.99965646  -0.02505733  -0.00768735 -14.7314117 ]
 [  0.02507203  -0.99968399  -0.00182186  -6.48518631]
 [ -0.00763927  -0.00201397   0.99996879 287.77684282]
 [  0.           0.           0.           1.        ]]


In [56]:
tf_err_in_m = np.linalg.inv(tf_m_in_cam) @ tf_s_in_c
print(tf_err_in_m)

vec = R.from_matrix(tf_err_in_m[:3,:3]).as_rotvec() 
print(vec/np.linalg.norm(vec))
print(np.linalg.norm(vec)*57)

[[ 0.9992722   0.03814551 -0.          0.06798593]
 [-0.03814551  0.9992722  -0.          2.2609825 ]
 [ 0.          0.          1.          0.00441573]
 [ 0.          0.          0.          1.        ]]
[ 0. -0. -1.]
2.174821780366478


In [57]:
keypoints3d_homog = np.hstack((
    keypoints,
    np.zeros((keypoints.shape[0],1)),
    np.ones((keypoints.shape[0],1))))

keypoints_corrected = (tf_err_in_m @ keypoints3d_homog.T).T

print(keypoints_corrected)

[[  7.63735315  39.50935466   0.00441573   1.        ]
 [ -4.63970504  39.97801042   0.00441573   1.        ]
 [ 28.3546594   29.86806545   0.00441573   1.        ]
 [-26.03172909  31.94417304   0.00441573   1.        ]
 [ 14.24399572  28.59940059   0.00441573   1.        ]
 [-12.058847    29.60346673   0.00441573   1.        ]
 [ 28.48950318  20.4500673    0.00441573   1.        ]
 [-26.88416624  22.56386266   0.00441573   1.        ]
 [ 38.50408988   6.3277775    0.00441573   1.        ]
 [-37.94622868   9.24613798   0.00441573   1.        ]
 [ 18.91316008 -25.75126375   0.00441573   1.        ]
 [-20.85787328 -24.2330724    0.00441573   1.        ]
 [ 19.33204213 -27.88579574   0.00441573   1.        ]
 [-21.43826343 -26.32945889   0.00441573   1.        ]]


In [ ]:
keypoints_new = (tf_s_in_c @ anchor_kpts.T).T
print(keypoints_new)
print(keypoints.shape)
keypoints = np.hstack((
    keypoints,
    np.zeros((keypoints.shape[0],1)),
    np.ones((keypoints.shape[0],1))))


keypoints_corrected = (tf_err_in_m @ keypoints.T).T

print(keypoints_corrected)

In [ ]:
anchor_kpts = np.zeros((6,2), dtype=float)
i = 0
anchor_kpts[2*i] = [6.143, 37.51]
anchor_kpts[2*i+1] = [-anchor_kpts[2*i][0], anchor_kpts[2*i][1]]
i +=1
anchor_kpts[2*i] = [33.75, 0]
anchor_kpts[2*i+1] = [-anchor_kpts[2*i][0], anchor_kpts[2*i][1]]
i+=1
anchor_kpts[2*i] = [19.9, -27.273]
anchor_kpts[2*i+1] = [-anchor_kpts[2*i][0], anchor_kpts[2*i][1]]

anchor_kpts = np.hstack((anchor_kpts, np.zeros((6,1))))
anchor_gt_kpts2D = np.array(
[(576, 266),
 (625, 267),
 (525, 349),
 (671, 353),
 (552, 409),
 (644, 409)], dtype=float)

In [ ]:
img_with_marker_name = 'marker.jpg'
img_without_marker_name = 'no_marker.jpg'
img = cv2.imread(img_without_marker_name)
utils.show_image_with_points(img,anchor_gt_kpts2D, "bb")
cv2.destroyAllWindows()

In [ ]:
obj_points = np.zeros((5,3), dtype=float)
obj_points[0][1] = 40.0
obj_points[1][0] = 40.0
obj_points[1][1] = 40.0
obj_points[2][0] = 40.0
obj_points[4][0] = 20.0
obj_points[4][1] = 20.0

print(obj_points.shape)

for r in results:
    imagePoints = r.corners.reshape(1,4,2)
    center = r.center.reshape(1,1,2)

    imgpoints = np.concatenate((imagePoints, center), axis=1)
    success, rvec, tvec = cv2.solvePnP(obj_points, imgpoints, mtx, dist)


In [ ]:
anchor_gt_2D = np.array(
[(623, 336),
 (610, 318),
 (636, 318),
 (597, 336),
 (649, 336),
 (610, 358),
 (636, 358)], dtype=float)

In [ ]:
obj_points = np.zeros((5,3), dtype=float)
obj_points[0][1] = 40.0
obj_points[1][0] = 40.0
obj_points[1][1] = 40.0
obj_points[2][0] = 40.0
obj_points[4][0] = 20.0
obj_points[4][1] = 20.0

print(obj_points)

In [ ]:
def draw_nums(img, points, show=False):
    for i in range(len(points)):
        point = (int(points[i][0][0]), int(points[i][0][1]))
        cv2.putText(img, str(i), point, cv2.FONT_HERSHEY_SIMPLEX, 12, (255,0,255))
        cv2.circle(img, point, 2, (255,0,255), -1)
    
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(10)

In [ ]:
def get_socket_points(img, points_array, lines_array, bbox_array, tvec, rvec, 
                      show=False, draw_l=True, draw_p=True, draw_b=True):
    if isinstance(img, str):
        img = cv2.imread(img)
#     img = increase_brightness(img, 50)
    img_points = None
    
    if draw_p:
        img_points, _ = cv2.projectPoints(points_array, rvec, tvec, mtx, dist)
        draw_points(img, img_points)
    if draw_l:
        lines_points, _ = cv2.projectPoints(lines_array, rvec, tvec, mtx, dist)
        draw_lines(img, lines_points)
    if draw_b:
        bbox_arr, _ = cv2.projectPoints(bbox_array, rvec, tvec, mtx, dist)
        draw_bbox(img, bbox_arr)
    
    if show:
        cv2.imshow("Image", img)
        cv2.waitKey(1)
    return img, img_points

In [ ]:
def detect_apriltag(img):
    if isinstance(img, str):
        print('reading img from file system')
        img = cv2.imread(img)
    if not type(img[0][0][0]) == np.uint8:
        img = img/img.max()
        img *= 255
        img = img.astype(np.uint8)
#         cv2.imshow('detect_apriltag', img)
        cv2.waitKey(1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    results = detector.detect(gray,
                              estimate_tag_pose=True,
                              camera_params=camera_params, 
                              tag_size=0.04)
    if not results:
        print('apriltag not found')
        return
    return results

In [ ]:
def warp_img(image, res):
    corners = res[0].corners
    
    corners[:2] = np.array(sorted(corners[:2], key=lambda x: x[1]))
    corners[2:] = np.array(sorted(corners[2:], key=lambda x: x[1]))

    a = corners[:2][:, 0].mean()
    c = corners[2:][:, 0].mean()
    b = corners[::2][:, 1].mean()
    d = corners[1::2][:, 1].mean()

    desired_corners = np.array([[a, b], [a, d], [c, b], [c, d]], dtype=int)

    tform = get_projection(desired_corners, corners)
    image_warped = warp(image, tform)
    return image_warped, desired_corners

In [ ]:
def get_projection(c_to, c_from):
    tform = ProjectiveTransform()
    tform.estimate(c_to, c_from)
    return tform

In [ ]:
# pose_left = ur10.getl()

In [ ]:
ur10.movej((-4.048973385487692, -1.1045106093036097, 1.546417236328125,
            2.811586380004883, -0.7216728369342249, 3.1041338443756104))

In [ ]:
# pose_center = ur10.getl()

In [ ]:
# pose_right = ur10.getl()

In [ ]:
points_arr = np.array(
    [[-132. ,   44. ,  -46. ],
       [-132. ,   27. ,  -46. ],
       [-143. ,   46. ,  -43. ],
       [-143. ,   40. ,  -45. ],
       [-142.5,   31. ,  -46. ],
       [-142.5,   24. ,  -45. ],
       [-149. ,   60. ,  -45. ],
       [-149. ,   43. ,  -44. ],
       [-149. ,   27. ,  -45. ],
       [-149. ,   11. ,  -45. ],
       [-156. ,   47. ,  -46. ],
       [-156. ,   39. ,  -45. ],
       [-156. ,   31. ,  -46. ],
       [-156. ,   23. ,  -46. ],
       [-162. ,   35. ,  -45. ],
       [-171. ,   44. ,  -46. ],
       [-171. ,   27. ,  -45. ]], dtype=np.float32)
DEPTH = -40
bbox = np.array([
    [10, 128, DEPTH],
    [59, 128, DEPTH],
    [59, 170, DEPTH],
    [10, 170, DEPTH]
], dtype=float)
bbox[:,[0, 1]] = bbox[:,[1, 0]]
bbox[:, 0] = -bbox[:, 0]

In [ ]:
lines = np.array([
    [0,0,0],
    [100,0,0],
    [0,100,0],
    [0,0,100]
], dtype=float)

In [ ]:
def draw_bbox(img, bbox_arr, show=False):
    print(bbox_arr)
    color = (0,255,0)
    cv2.line(img, 
            [int(x) for x in bbox_arr[0][0]],
            [int(x) for x in bbox_arr[1][0]], color, 1)
    cv2.line(img, 
            [int(x) for x in bbox_arr[1][0]], 
            [int(x) for x in bbox_arr[2][0]], color, 1)
    cv2.line(img, 
            [int(x) for x in bbox_arr[2][0]], 
            [int(x) for x in bbox_arr[3][0]], color, 1)
    cv2.line(img, 
            [int(x) for x in bbox_arr[3][0]], 
            [int(x) for x in bbox_arr[0][0]], color, 1)
    
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(10)

In [ ]:
def draw_lines(img, line_arr, show=False):
    colors = [(255,255,0), (0,255,0), (0,0,255)]
    for i, line in enumerate(line_arr[1:]):
        cv2.line(img, [int(x) for x in line_arr[0][0]], [int(x) for x in line[0]], colors[i-1], 2)
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(10)

In [ ]:
cv2.destroyAllWindows()

In [ ]:
name = 2
count = 0
centers = []
# ress = []
while count < 10:
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
    count += 1
    
    if not res: 
        continue

#     ress = res
#     break
#     centers.append(res[0].center)
#     print(res)
#     ress.append(res)
    
    draw_apriltag(img, res,show=True)
    tvec, rvec,objpts = solve_pnp(res)
    print(tvec)
#     processed, _ = get_socket_points(img, points_arr, lines, bbox, tvec, rvec, 
#                                    show=True, draw_l=True, draw_p=True, draw_b=False)
    
    
#     fname = f'/home/viacheslav/jupyter_notebooks/{name}_processed.jpg'
#     cv2.imwrite(fname, img)
#     name += 1
#     break


In [ ]:
[[-23.50502963]
 [  8.35980833]
 [110.64977061]]

In [ ]:
centers = np.array(centers)

print(np.var(centers,axis = 0))

In [ ]:
name = 1
ress1 = []
centers = []
corners = []
count = 0
while name< 100:
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
#     print(res)
    if not res: 
        continue
    for r in res:
        centers.append(r.center)
        corners.append(r.corners)
    count+=1
#     ress.append(res)
#     cv2.imshow("Image", img)
#     cv2.waitKey(10)
    draw_apriltag(img, res)
#     tvec, rvec, objpts = solve_pnp(res)
#     processed, _ = get_socket_points(img, points_arr, lines, bbox, tvec, rvec, 
#                                    show=True, draw_l=True, draw_p=False, draw_b=False)
    
    
    fname = f'/home/viacheslav/jupyter_notebooks/new_data/{name}_processed.jpg'
#     print(fname)
    cv2.imwrite(fname, img)
    name += 1
#     break


In [ ]:
centers = np.array(centers)
print(np.mean(centers,axis = 0))
print(np.sqrt(np.var(centers,axis = 0)))

In [ ]:
corners = np.array(corners)
# i = 0
for i in range (4):
# print(np.mean(corners[:,i],axis = 0))
    print(np.sqrt(np.var(corners[:,i],axis = 0)))
# print(corners[0])

In [ ]:
corners[:10,0]

In [ ]:
# Good lighting 
print(np.mean(centers,axis = 0))
print(np.var(centers,axis = 0))

[655.96844896 321.05286757]
[0.00012992 0.00013354]

# Poor lighting 
[656.07557452 321.20941614]
[0.00089831 0.00115515]



In [ ]:
centers

In [ ]:
ress1[0][0]

In [ ]:
ress[0][0].center

In [ ]:
results = np.array([r[0].center for r in ress])
results[:,1].std()

In [ ]:
def make_photo(path='/home/viacheslav/jupyter_notebooks/processed_23_9', name=0):
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
    if not res: 
        return name
    draw_apriltag(img, res)
    tvec, rvec = solve_pnp(res)
    processed = get_socket_points(img, points_arr, lines, bbox, tvec, rvec, draw_b=False, draw_l=False)
    cv2.imshow('img', img)
    cv2.waitKey(1)
    fname = path+f'/{name}_processed.jpg'
    cv2.imwrite(fname, processed)
    name += 1
    return name

In [ ]:
def move_to_next_pos_and_photo(end:list, num_photos:int, img_name):
    start = ur10.getl()
    deltas = np.array([x-y for x, y in zip(end, start)])/num_photos
    for _ in range(num_photos):
        ur10.movel(ur10.getl()+deltas, vel=0.1, acc=0.5)
        img_name = make_photo(name=img_name)
    return img_name

In [ ]:
cv2.destroyAllWindows()

In [ ]:
ur10.getl()

In [ ]:
boundary_positions = np.array([[ 0.232085  , -0.61961176,  0.43073168,  1.13699011,  1.24920145,
         1.22539067],
       [ 0.2333736 , -0.60270578,  0.63426659,  1.17489684,  1.22881322,
         1.24945387],
       [ 0.42691084, -0.61264868,  0.58616166,  1.3315313 ,  1.41684329,
         1.23695536],
       [ 0.29272549, -0.60669106,  0.33210261,  0.9753124 ,  0.95871666,
         1.43494421],
       [ 0.33165193, -0.25687359,  0.36506057,  1.30079001,  0.75824351,
         0.868352  ],
       [ 0.33153143, -0.22721475,  0.82933533,  1.67432401,  0.98844306,
         0.83238533],
       [ 0.44529088, -0.30760203,  0.6630999 ,  1.49787613,  1.12812965,
         0.9164792 ],
       [ 0.48105068, -0.37105609,  0.34244749,  1.08186592,  0.84939581,
         1.07530735]])

In [ ]:
min_x, max_x = boundary_positions[:, 0].min(), boundary_positions[:, 0].max()
min_y, max_y = boundary_positions[:, 1].min(), boundary_positions[:, 1].max()
min_z, max_z = boundary_positions[:, 2].min(), boundary_positions[:, 2].max()
min_rx, max_rx = boundary_positions[:, 3].min(), boundary_positions[:, 3].max()
min_ry, max_ry = boundary_positions[:, 4].min(), boundary_positions[:, 4].max()
min_rz, max_rz = boundary_positions[:, 5].min(), boundary_positions[:, 5].max()

In [ ]:
img_name = 0

In [ ]:
while True:
    next_pos = [np.random.uniform(min_x, max_x),
                np.random.uniform(min_y, max_y),
                np.random.uniform(min_z, max_z),
                np.random.uniform(min_rx, max_rx),
                np.random.uniform(min_ry, max_ry),
                np.random.uniform(min_rz, max_rz)]
    img_name = move_to_next_pos_and_photo(next_pos, 50, img_name)

In [ ]:
pose_left = [0.4144699791608615,
 -0.28471817373934244,
 0.4506025223319278,
 1.3698459362193829,
 0.8751222887301229,
 0.7944053357672337]

In [ ]:
pose_center = [0.4168558559693215,
 -0.5621571861068889,
 0.4414088449025262,
 1.1886706848218183,
 1.2237760813323468,
 1.1861928832927722]

In [ ]:
pose_right = [0.613646044924774,
 -0.8457833066867604,
 0.39668770616099447,
 0.86630043677316,
 1.6128758106937202,
 1.7491595656007006]

In [ ]:
def detect_red_pixels(path:str, img:np.array=None):
    red_pixel =  np.array([237,28, 36])
#     up=220
#     low=40
    if img is None:
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    indices = []
    for x, row in enumerate(img):
        for y, cell in enumerate(row):
            if np.array_equal(img[x, y], red_pixel):
#             if img[x, y,0] > up and img[x,y,1] < low and img[x,y,2] < low:
                indices.append((y,x))
#     assert len(indices) == 17, f'wrong keypoint number!! {indices}'
    return indices

In [ ]:
cv2.destroyAllWindows()

# Get new keypoints from image

In [ ]:
img = cv2.imread('/home/viacheslav/jupyter_notebooks/no_marker.jpg')

In [ ]:
px = np.array(detect_red_pixels('/home/viacheslav/jupyter_notebooks/Screenshot_1.png'))
px

In [ ]:
cv2.imshow('a', img)
cv2.waitKey(0)

In [ ]:
cv2.imshow('a', cv2.imread('/home/viacheslav/jupyter_notebooks/marker.jpg'))
cv2.waitKey(0)

In [ ]:
res = detect_apriltag('/home/viacheslav/jupyter_notebooks/marker.jpg')[0]
res

In [ ]:
draw_corners(img, res, True)

In [ ]:
plt.scatter(px[:,0], -px[:,1],c=range(len(px)))

In [ ]:
dx = np.array(((res.corners[2] - res.corners[1]) + (res.corners[3] - res.corners[0]))/12).T
dy = np.array(((res.corners[2] - res.corners[3]) + (res.corners[1] - res.corners[0]))/12).T
mat = np.vstack([dx, dy])

In [ ]:
mat1 = np.linalg.inv(mat)

In [ ]:
marker_points = []
det = np.linalg.det(mat1)
for point in px:
    kp = point.copy()
#     kp[0] -= int(res.center[0])
#     kp[1] -= int(res.center[1])
    p = [kp[0]*mat1[0, 0]+kp[1]*mat1[1, 0], kp[0]*mat1[0,1]+kp[1]*mat1[1,1]]
#     p = np.array([kp[0]*dy[1] - kp[1]*dy[0], -kp[0]*dx[1]+kp[1]*dx[0]])
#     p /= det
    
    marker_points.append(p)
marker_points = np.array(marker_points)

In [ ]:
marker_points

In [ ]:
plt.scatter(marker_points[:, 0], -marker_points[:, 1], c=range(len(marker_points)))

In [ ]:
draw_points_on_aruco(img, [res], marker_points, True)
cv2.waitKey(0)

In [ ]:
import annotation_utils
def draw_bbox_by_points(img, 
                        res, 
                        points=annotation_utils.get_points_from_CAD(),
                        show=False):
    new_points = annotation_utils.get_keypoints_on_image(img, points, camera_params)
    top = new_points[:, 0].min()
    bottom = new_points[:, 0].max()
    left = new_points[:, 1].min()
    right = new_points[:, 1].max()
    bbox_corners = np.array([
        [top, left],
        [top, right],
        [bottom, right],
        [bottom, left],
    ])
    if show:
        imgc = img.copy()
        cv2.line(imgc, bbox_corners[0], bbox_corners[1], (0,255,0), 1)
        cv2.line(imgc, bbox_corners[1], bbox_corners[2], (0,255,0), 1)
        cv2.line(imgc, bbox_corners[2], bbox_corners[3], (0,255,0), 1)
        cv2.line(imgc, bbox_corners[3], bbox_corners[-1], (0,255,0), 1)
        cv2.imshow('draw_bbox_by_points', imgc)
        cv2.waitKey(0)
    return bbox_corners

In [ ]:
img = cv2.imread('/home/viacheslav/jupyter_notebooks/data/far_photos/20.jpg')
res = annotation_utils.detect_apriltag('/home/viacheslav/jupyter_notebooks/data/far_photos_with_marker/20.jpg', camera_params)
draw_bbox_by_points(img, res, show=True)

### get robot poses

In [ ]:
inserted_j = ur10.getj()
inserted_l = ur10.getl()
print(inserted_j)
print(inserted_l)

In [ ]:
ur10.movel(zero_pose_l)

In [ ]:
zero_pose_l = ur10.getl()
zero_pose_j = ur10.getj()
print(zero_pose_l)
print(zero_pose_j)

In [ ]:
zero_pose_l = [0.6053738400708184,
 -0.5532917677103614,
 0.5293931050569378,
 1.242668373605403,
 1.2235408761040583,
 1.2353307425198223] # 15 cm from socket
ur10.movel(zero_pose_l)

In [ ]:
ur10.movel([zero_pose_l[0]-0.5, *zero_pose_l[1:]], vel=1)
# ur10.movej([zero_pose_l[0]-0.5, *zero_pose_l[1:]], vel=1)

In [ ]:
far_pose_l = ur10.getl()
far_pose_j = ur10.getj()
print(far_pose_l)
print(far_pose_l)

In [ ]:
x_far = [0, .25]
y_far = [-.25, .25]
z_far = [-.2, .2]

In [ ]:
x_close = [-.1, -.25]
y_close = [-.10, .10]
z_close = [-.15, .05]

In [ ]:
ur10.movel([far_pose_l[0], far_pose_l[1]+y_far[0], far_pose_l[2]+z_far[1], 
            *far_pose_l[3:]], vel=1)

In [ ]:
ur10.movel(zero_pose_l)

In [ ]:
ur10.movel([zero_pose_l[0]+x_close[0], zero_pose_l[1]+y_close[0], zero_pose_l[2]+z_close[1], 
            *zero_pose_l[3:]], vel=1)

In [ ]:
R.from_rotvec([ 1.2186813991654113,
 1.2029606765250755,
 1.1943521654058413]).as_rotvec()

In [ ]:
from scipy.spatial.transform import Rotation as R

# angles along X and Y
orients = np.array([
#         [-5, -5],
    [-5, 0],
#     [-5, 5],
#     [0,  -5],
    [0,  0],
#     [0,  5],
#     [5,  -5],
    [5,  0],
#     [5, 5]
], dtype=float)

# orients *= np.pi/180
rotations = [R.from_euler('xy', x, degrees=True) for x in orients]

In [ ]:
r0 = rotations[0]
r0.as_matrix()

In [ ]:
r0.apply(lrc[3:])

In [ ]:
  img_coords = {}

In [ ]:
img_coords

In [ ]:
ur10.movel(zero_pose)

In [ ]:
llc

## Record points with aruco on socket

In [ ]:
ur10.movel([far_pose_l[0]+x_far[0], far_pose_l[1]+y_far[0], far_pose_l[2]+z_far[1], 
            *far_pose_l[3:]])
#            , vel=1)
# ur10.movel(far_pose_l)

In [ ]:
img_coords = [0]*2000
!rm data/points/*
!rm data/no_points/*

In [ ]:
ur10.movel(lrf, vel=1., acc=1.)

In [ ]:
img_coords = [0]*600
# delta = .05
img_name = -1
js = []
# ur10.movel([far_pose_l[0]+x_far[0], far_pose_l[1]+y_far[0], far_pose_l[2]+z_far[0], 
#             *far_pose_l[3:]], vel=1)
pose = zero_pose_l.copy()
# i, j, k = 0,0,0
# these ranges for far
# xrange = [0, .05, .1, .15, .2, .25]
# yrange = [-.25, -.2, -.15, -.1, -.05, 0, .05, .1, .15, .2, .25]
# zrange = [-.2, -.15, -.1, -.05, 0, .05, .1, .15, .2] 

#these for close
xrange = [-.1, -.125, -.15, -.175, -.2, -.225, -.25]
yrange = [-.1, -.075, -.05, -.025, 0, .025, .05, .1, ]
zrange = [-.15, -.1, -.05, 0, .05,] 

from itertools import product
poss = list(product(*[xrange, yrange, zrange]))

for pos in poss:
#     print(pose, pos)
    ur10.movel([pose[0]+pos[0], pose[1]+pos[1], pose[2]+pos[2], *pose[3:]])
#             pose = [pose[0]+delta*i, pose[1]+delta*j, pose[2]+delta*k, *pose[3:]]
    img_name +=1
    img = get_photo_from_realsense()
    cv2.imshow('robot sees', img)
    cv2.waitKey(1)
    cv2.imwrite(
            f'/home/viacheslav/jupyter_notebooks/data/far_photos/{img_name}.jpg',
            img)
    js.append(ur10.getj())
            
json.dump(js, open(f'/home/viacheslav/jupyter_notebooks/data/far_photos_joints.json', 'w'))

In [ ]:
ur10.movel(far_pose_l, vel=1)

In [ ]:
img_name = -1
for posej in tqdm(js):
    ur10.movej(posej)
    img_name +=1
    img = get_photo_from_realsense()
    cv2.imshow('robot sees', img)
    cv2.waitKey(1)
    cv2.imwrite(
            f'/home/viacheslav/jupyter_notebooks/data/far_photos_with_marker/{img_name}.jpg',
            img)

In [ ]:
img_coords = [0]*600
delta = .05
img_name = -1
js = []
ur10.movel([far_pose_l[0]+x_far[0], far_pose_l[1]+y_far[0], far_pose_l[2]+z_far[0], 
            *far_pose_l[3:]], vel=1)
for i in range(6):
    for j in tqdm(range(11)):
        for k in range(7):
            for l in range(len(rotations)):
                pose = [lrf[0]+delta*i, lrf[1]+delta*j, lrf[2]+delta*k, *lrf[3:]]
                d_angle = orients[l]
                
                init_rotvec = lrc[3:]
                matrix = R.from_rotvec(init_rotvec).as_matrix()
                pose[3:] = R.from_matrix(rotations[l].apply(matrix)).as_rotvec()
                
                ur10.movel(pose, vel=1)
            
                img = get_photo_from_realsense()
                
                cv2.imwrite(
                    f'/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos_2/{img_name}.jpg',
                    img)

#                 res = detect_apriltag(img)

#                 if res: 
#                     img_w, desired_corners = warp_img(img, res)

#                     res_warp = detect_apriltag(img_w)
#                     if res_warp:
#                         img_name += 1
#                         center, points_w = draw_points_on_aruco(img_w, res_warp, seventeen_points, False)

#                         inv_project = get_projection(res[0].corners, desired_corners)
#                         img_ok = warp(img_w, inv_project)
#                         points = np.array([inv_project.params @ np.array([p[0], p[1], 1]) for p in points_w])
            
#                         center, points = draw_points_on_aruco(img, res, seventeen_points, True)

#                         img_coords[img_name] = points
#                         js.append(ur10.getj())
                
                        

# json.dump(img_coords, open(f'/home/viacheslav/jupyter_notebooks/data/seventeen_2.json', 'w'))
# json.dump(js, open(f'/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos_2.json', 'w'))

In [ ]:
img_name = -1
for posej in tqdm(js):
    ur10.movej(posej)
    img_name +=1
    img = get_photo_from_realsense()
    cv2.imshow('robot sees', img)
    cv2.waitKey(1)
    cv2.imwrite(
            f'/home/viacheslav/jupyter_notebooks/data/far_photos_with_marker/{img_name}.jpg',
            img)

In [ ]:
json.dump(img_coords, open(f'/home/viacheslav/jupyter_notebooks/data/seventeen_2.json', 'w'))
json.dump(js, open(f'/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos_2.json', 'w'))

In [ ]:
img_coords = [0]*2000

for i, posej in enumerate(js):
    ur10.movej(posej, vel=1, acc=1)
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
            
    center, points = draw_points_on_aruco(img, res, seventeen_points, True)

    img_coords[i] = points
    
    
with open('img_coords.pickle', 'wb') as handle:
    pkl.dump(img_coords, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [ ]:

with open('img_coords.pickle', 'wb') as handle:
    pkl.dump(img_coords, handle, protocol=pkl.HIGHEST_PROTOCOL)
    
# with open('js.pickle', 'wb') as handle:
#     pkl.dump(js, handle, protocol=pkl.HIGHEST_PROTOCOL)

In [ ]:

# approved by Viktor
def draw_points_on_aruco(img, res, changes=None, show=False):
    dx = (res[0].corners[2] - res[0].corners[1])/6
    dy = (res[0].corners[2] - res[0].corners[3])/6
    print (dx*dy)
    
    if changes is None:
        changes = np.array([
            [0, 0],
            [3, 1],
            [2, 2],
            [-2, 2],
            [-3, 1],
            [-3, -1],
            [-2, -3],
            [2, -3],
            [3, -1]
        ])
        
    points = np.zeros((changes.shape[0], 2), dtype=float)
    for i, ch in enumerate(changes):
        points[i] = [ch[0]*dx[0]+ch[1]*dy[0], ch[0]*dx[1]+ch[1]*dy[1]]
        
    points[:, 0] += sum([x[0] for x in res[0].corners])/4
    points[:, 1] += sum([x[1] for x in res[0].corners])/4
    
    if show:
#         draw_corners(img, res[0])
        for p in points:
            kp = int(p[0]), int(p[1])
            cv2.circle(img, kp, 2, (0, 0, 255), -1)
        cv2.imshow('draw_points_on_aruco', img)
        cv2.waitKey(1)
    return points[0], points[1:]

def warp_img(image, res):
    corners = res[0].corners
    
    corners[:2] = np.array(sorted(corners[:2], key=lambda x: x[1]))
    corners[2:] = np.array(sorted(corners[2:], key=lambda x: x[1]))

    a = corners[:2][:, 0].mean()
    c = corners[2:][:, 0].mean()
    b = corners[::2][:, 1].mean()
    d = corners[1::2][:, 1].mean()

    desired_corners = np.array([[a, b], [a, d], [c, b], [c, d]], dtype=int)

    tform = get_projection(desired_corners, corners)
    image_warped = warp(image, tform)
    return image_warped, desired_corners, tform

def draw_corners(img, r, show=False):
    if isinstance(r, np.ndarray):
        imagePoints = [r]
    else:
        imagePoints = r.corners.reshape(1,4,2)
    
    for corner in range(np.size(imagePoints[0],axis=0)):
        center = ((int(imagePoints[0][corner][0]),int(imagePoints[0][corner][1])))
        cv2.circle(img, center, 3, (0,0,255), -1)
    
    if show:
        cv2.imshow('img', color_image)
    return imagePoints[0]

In [ ]:
def draw_corners(img, r, show=False):
    if isinstance(r, np.ndarray):
        imagePoints = [r]
    else:
        imagePoints = r.corners.reshape(1,4,2)
    
    for corner in range(np.size(imagePoints[0],axis=0)):
        center = ((int(imagePoints[0][corner][0]),int(imagePoints[0][corner][1])))
        cv2.circle(img, center, 3, (0,0,255), -1)
    
    if show:
        cv2.imshow('img', img)
        cv2.waitKey(0)
    return imagePoints[0]

In [ ]:
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()
# buf_sz = 10
# buffer = np.zeros((buf_sz, 17, 3), dtype=float)
# index = 0
while True:
    index += 1
    index = index % buf_sz
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
    cv2.imshow('original', img)
    cv2.waitKey(1)
    if res: 
        img_w, desired_corners,tform = warp_img(img, res)
#         desired corners are int!
#         print("desired_corners",desired_corners)
        
        res_warp = detect_apriltag(img_w)
        if res_warp:
            center, points_w = draw_points_on_aruco(img_w, res_warp, seventeen_points, True)
#             corners_w = draw_corners(img_w, res_warp[0], False)
            inv_project = get_projection(res[0].corners, desired_corners)
#             print(inv_project)
            str_project = get_projection(desired_corners, res[0].corners)
            img_ok = warp(img_w, inv_project)
            
#             points = [np.linalg.inv(tform.params) @ np.array([p[0], p[1], 1]) for p in points_w]
            points = [str_project.params @ np.array([p[0], p[1], 1]) for p in points_w]

            for p in points:
                p/= p[2]
            points = np.array(points)
            
#             points = np.array([inv_project.params @ np.array([p[0], p[1], 1]) for p in points_w])
#             buffer[index] = points
#             corners = np.array([inv_project.params @ np.array([p[0], p[1], 1]) for p in corners_w])
#             print('corners', corners)
#             for p in np.mean(buffer, axis=0):
#                 kp = int(p[0]), int(p[1])
#                 cv2.circle(img_ok, kp, 2, (0,255, 255), -1)
            for c in points:
                kc = int(c[0]), int(c[1])
                cv2.circle(img, kc, 3, (0,255, 255), -1)
            cv2.imshow('final', img)
#             cv2.waitKey(10)
#             center, points = draw_points_on_aruco(img, res, seventeen_points, True)

## Save data

In [ ]:
json.dump(img_coords, open(f'/home/viacheslav/jupyter_notebooks/data/seventeen.json', 'w'))
json.dump(js, open(f'/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos.json', 'w'))

## Get photos

In [ ]:
coords_j = json.load(open('/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos.json'))
print('data len', len(coords_j))
for i, position in tqdm(enumerate(coords_j)):
    ur10.movej(position, vel=1, acc=1)
    img = get_photo_from_realsense()
    cv2.imwrite(f'/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos_17/{i}.png', img)

In [ ]:
json.dump( img_coords, open(f'/home/viacheslav/jupyter_notebooks/data/seventeen.json', 'w'))

In [ ]:
ur10.movej([-4.250930611287252, -0.8361623922931116, 1.6090149879455566, 2.2111873626708984, -0.48303395906557256, 3.2748172283172607],)

In [ ]:

llc_points = []
lrc_points = []
for _ in range(10):
    ur10.movel(llc)
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
    center, points = draw_points_on_aruco(img, res, seventeen_points, True)
    lrc_points.append(points)
    ur10.movel(lrc)
    img = get_photo_from_realsense()
    res = detect_apriltag(img)
    center, points = draw_points_on_aruco(img, res, seventeen_points, True)
    llc_points.append(points)

In [ ]:
points

In [ ]:
cv2.destroyAllWindows()

In [ ]:

points = json.load(open('/home/viacheslav/jupyter_notebooks/data/seventeen.json'))
# cv2.imshow('img', img)
# cv2.waitKey(0)
for i in range(len(points)): 
    img = cv2.imread(f'/home/viacheslav/jupyter_notebooks/data/joint_coords_for_photos_17/{i}.png')    
    draw_points(img, points[i], True, True)

In [ ]:
pose = ur10.getj()

In [ ]:
ur10.movej(pose)

# Robot sees

## Get nice graphs

# Record points